In [63]:
# Basic imports
import pandas as pd
import numpy as np


import sqlalchemy as sa

import warnings
warnings.filterwarnings('ignore')
import os
from dotenv import load_dotenv
load_dotenv()

engine = sa.create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
    .format(host=os.getenv('host'), db=os.getenv('db'), user=os.getenv('uname'), pw=os.getenv('password')))

engine2 = sa.create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
    .format(host=os.getenv('host'), db=os.getenv('db2'), user=os.getenv('uname'), pw=os.getenv('password')))



In [64]:
def add_zeros(arr):
    return np.array([s.zfill(3) for s in arr])

q = """SELECT barings_data.*, borrower7afoia.BusinessOrgCd, borrower7afoia.WomanOwnerInd, borrower7afoia.VetCd, borrower7afoia.UseOfProceedCountyCd, borrower7afoia.UrbanRuralInd,
round(1-loantbl7afoia.BankParticipationPct,2) as `Gty Pct`, round(loantbl7afoia.BankInterestPct,2) as `Lender Rate`,
elipsmisc7afoia.EmployeesExistingQty FROM barings_data
LEFT JOIN borrower7afoia on borrower7afoia.ObservationNmb = barings_data.GP
LEFT JOIN loantbl7afoia on loantbl7afoia.ObservationNmb = barings_data.GP
LEFT JOIN elipsmisc7afoia on elipsmisc7afoia.ObservationNmb  = barings_data.GP;"""

df = pd.read_sql(q,con=engine)
df['UseOfProceedCountyCd'] = df['UseOfProceedCountyCd'].astype(str)
df['UseOfProceedCountyCd'] = add_zeros(df['UseOfProceedCountyCd'])

df.head()

,GP,Note_Yr,Note_Mth,NoteDt,LoanAmt,MaturityDt,MaturityMthsQty,PrepayDt,PrepayMthsQty,DefaultDt,...,state_abbreviation,state_name,BusinessOrgCd,WomanOwnerInd,VetCd,UseOfProceedCountyCd,UrbanRuralInd,Gty Pct,Lender Rate,EmployeesExistingQty
0,423883,2000,3,2000-03-31,25000.0,2007-03-31,84,None,NaN,None,...,CA,California,1.0,2,1.0,037,0.0,0.5,0.13,1.0
1,423886,2000,5,2000-05-31,20000.0,2007-05-31,84,2000-06-30,0.0,None,...,PA,Pennsylvania,1.0,2,1.0,003,0.0,0.5,0.13,2.0
2,423917,2000,7,2000-07-31,10000.0,2001-07-31,12,None,NaN,None,...,MA,Massachusetts,3.0,1,1.0,017,0.0,0.5,0.12,3.0
3,423941,2000,7,2000-07-31,100000.0,2001-07-31,12,None,NaN,None,...,MA,Massachusetts,3.0,2,1.0,017,0.0,0.5,0.10,1.0
4,423946,2002,5,2002-05-31,65000.0,2007-05-31,60,2002-06-30,0.0,None,...,NC,North Carolina,3.0,1,1.0,119,0.0,0.8,0.10,7.0


In [82]:
df = df.rename(columns={'CodeText':'IndustryDesc','Code':'NAICS'})

In [87]:
df = df.rename(columns={'PP_qty':'MoB'})

1. Resolve County Codes

In [65]:
df['use_of']=df['state_abbreviation']+df['UseOfProceedCountyCd']
state_q="""SELECT * FROM STATE_COUNTY_CODES"""
state_county = pd.read_sql(state_q,con=engine2)
state_county['use_of'] = state_county['StateAbbreviation'] + state_county['CountyCode']
df['use_of'] = df['use_of'].str.strip()
county_map = state_county[['use_of','CountyName']].set_index('use_of').to_dict()

df['CountyName'] = df['use_of'].map(county_map['CountyName'])

df['CountyName'].value_counts()

CountyName
LOS ANGELES     34492
ORANGE          15306
MARICOPA        12256
HARRIS          11866
COOK            11043
                ...  
HARTLEY             1
TREASURE            1
THROCKMORTON        1
Eastern             1
FOARD               1
Name: count, Length: 1918, dtype: int64

2. Resolve Woman Codes

In [66]:
woman = pd.read_sql(sql="""SELECT * FROM womancode_f75tbl""", con=engine2)

woman.set_index('WomanCode',inplace=True)

woman_map = woman.to_dict()['WomanCodeDesc']

df['WomanOwnerInd'] = df['WomanOwnerInd'].astype(int).astype(str)

df['WomanDesc'] = df['WomanOwnerInd'].map(woman_map)


3. Resolve Vet Codes

In [67]:
veteran = pd.read_sql(sql="""SELECT * FROM veteranloan_f76tbl""", con=engine2)

veteran.set_index('VetCode',inplace=True)

veteran_map = veteran.to_dict()['VetCodeDesc']

df['VetCd'] = df['VetCd'].fillna(1)

df['VetCd'] = df['VetCd'].astype(int).astype(str)

df['veteranDesc'] = df['VetCd'].map(veteran_map)

In [68]:
df['veteranDesc'].value_counts()

veteranDesc
Non-Veteran            738226
Other Veteran           38435
Vietnam Era Veteran     11027
Name: count, dtype: int64

4. Resolve Business Org Codes

In [72]:
bus_org = pd.read_sql(sql="""SELECT * FROM busiorgcode_f68tbl""", con=engine2)

bus_org.set_index('OrgCode',inplace=True)

busi_org_map = bus_org.to_dict()['OrgCodeDescription']

df['BusinessOrgCd'] = df['BusinessOrgCd'].fillna(1)

df['BusinessOrgCd'] = df['BusinessOrgCd'].astype(int).astype(str)

df['BusinessOrgDesc'] = df['BusinessOrgCd'].map(busi_org_map)

In [86]:
print((df.columns).to_list())

['GP', 'Note_Yr', 'Note_Mth', 'NoteDt', 'LoanAmt', 'MaturityDt', 'MaturityMthsQty', 'PrepayDt', 'PrepayMthsQty', 'DefaultDt', 'DefaultMthsQty', 'PP_qty', 'NAICS', 'IndustryDesc', 'Margin', 'state_abbreviation', 'state_name', 'BusinessOrgCd', 'WomanOwnerInd', 'VetCd', 'UseOfProceedCountyCd', 'UrbanRuralInd', 'Gty Pct', 'Lender Rate', 'EmployeesExistingQty', 'use_of', 'CountyName', 'WomanDesc', 'veteranDesc', 'BusinessOrgDesc']


In [93]:
df = df[['GP', 'Note_Yr', 'Note_Mth', 'NoteDt', 'MaturityDt', 'MaturityMthsQty', 'LoanAmt', 'Margin', 'Lender Rate', 'Gty Pct', 'PrepayDt', 'PrepayMthsQty', 'DefaultDt', 'DefaultMthsQty', 'MoB', 'NAICS', 'IndustryDesc', 'state_abbreviation', 'state_name', 'BusinessOrgCd', 'WomanOwnerInd', 'WomanDesc', 'VetCd', 'veteranDesc', 'use_of', 'UseOfProceedCountyCd', 'CountyName', 'EmployeesExistingQty', 'BusinessOrgDesc']]

In [96]:
df.to_sql('barings_data_v2', con=engine, index=False, if_exists='replace')

795113

In [98]:
q="""SELECT * FROM barings_data_v2"""

df1 = pd.read_sql(sql=q, con=engine)

In [99]:
df1.to_csv('presentable_loantapes/barings_SBA7(a)_data.csv')